In [6]:
from PIL import Image, ImageOps  # Pillow 모듈
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms  # torchvision 관련 모듈

import pandas as pd
import h5py  # HDF5 파일 형식을 위한 라이브러리

import numpy as np


### 크기 조정

크기 확인

In [ ]:
# 이미지 파일 경로 지정
image_path = r'C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset\image\EmotionROI\images\anger\1.jpg'

# 이미지 열기
image = Image.open(image_path)

# 이미지 크기 확인
width, height = image.size
print(f"Image size: {width} x {height} pixels")


##### 패딩

In [ ]:
# 원하는 정사각형 크기 지정 
desired_size = 512

# 패딩을 추가하여 지정한 크기의 정사각형으로 변환
padded_image = ImageOps.pad(image, (desired_size, desired_size), color=(0, 0, 0))  # (0, 0, 0)은 검은색 패딩
padded_image.show()

In [ ]:
# 이미지가 있는 폴더 경로
input_folder = r'C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset\image\EmotionROI\ground_truth\surprise'  # 원본 이미지 폴더 경로
output_folder = r'C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset_preprocessed\EmotionROI_padding\ground_truth\surprise'  # 결과를 저장할 폴더 경로

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 원하는 정사각형 크기 지정 (예: 512x512)
desired_size = 512

# 폴더 내 모든 JPG 이미지에 대해 반복 실행
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.jpg'):  # JPG 확장자만 처리
        image_path = os.path.join(input_folder, filename)
        
        # 이미지 열기
        image = Image.open(image_path)
        
        # 패딩을 추가하여 지정한 크기의 정사각형으로 변환
        padded_image = ImageOps.pad(image, (desired_size, desired_size), color=0)  
        
        # 결과 이미지를 저장
        output_path = os.path.join(output_folder, filename)
        padded_image.save(output_path)

print("전처리 완료")

##### 중앙 크롭

In [ ]:
# 중앙 크롭
cropped_image = ImageOps.fit(image, (512, 512), method=0, bleed=0.0, centering=(0.5, 0.5))
cropped_image.show()

In [ ]:
## 데이터셋 만들기 - images ##

# 이미지가 있는 폴더 경로
input_folder = r'C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset\EmotionROI\ground_truth\surprise'  # 원본 이미지 폴더 경로
output_folder = r'C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset_preprocessed\EmotionROI_crop\ground_truth\surprise'  # 결과를 저장할 폴더 경로

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 원하는 정사각형 크기 지정 (예: 512x512)
desired_size = 512

# 폴더 내 모든 JPG 이미지에 대해 반복 실행
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.jpg'):  # JPG 확장자만 처리
        image_path = os.path.join(input_folder, filename)
        
        # 이미지 열기
        image = Image.open(image_path)
        
        # 중앙 크롭을 수행하여 지정한 크기의 정사각형으로 변환
        cropped_image = ImageOps.fit(image, (desired_size, desired_size), method=0, bleed=0.0, centering=(0.5, 0.5))
        
        # 결과 이미지를 저장
        output_path = os.path.join(output_folder, filename)
        cropped_image.save(output_path)

print("전처리 완료")

### 데이터셋 로드

In [3]:
# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VATT 모델에 맞게 이미지 크기 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 정규화 값
])

# ImageFolder를 사용하여 데이터셋 로드
image_folder_path = r"C:\project_files\Multimodal_Auto _BGM_Addition_System\dataset_preprocessed\EmotionROI_crop\images"  # 데이터셋 폴더 경로
image_dataset = datasets.ImageFolder(root=image_folder_path, transform=transform)

# 데이터 로더 생성
image_dataloader = DataLoader(image_dataset, batch_size=32, shuffle=True)

# 클래스 인덱스 확인
print(image_dataset.class_to_idx)  

{'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}


### 임베딩

In [7]:
# 감정 레이블 매핑
emotion_label_mapping = {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}

# 디바이스 설정 (GPU 사용 가능 시 GPU로, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 이미지 전처리 설정 (이미지를 224x224 크기로 변환하고, 텐서로 변환)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지 폴더에서 이미지와 레이블을 배치 단위로 로드하는 함수
def load_images_in_batches(image_folder_path, batch_size):
    images = []
    labels = []
    
    for emotion, label in emotion_label_mapping.items():
        folder_path = os.path.join(image_folder_path, emotion)
        for img_name in os.listdir(folder_path):
            if img_name.endswith('.jpg'):  # 이미지 파일만 처리
                img_path = os.path.join(folder_path, img_name)
                image = Image.open(img_path)
                image = preprocess(image)
                images.append(image)
                labels.append(label)
                
                # 배치 크기만큼 모이면 처리
                if len(images) == batch_size:
                    images_tensor = torch.stack(images).to(device)
                    labels_tensor = torch.tensor(labels).to(device)
                    yield images_tensor, labels_tensor
                    images = []
                    labels = []
    
    # 남은 이미지 처리
    if len(images) > 0:
        images_tensor = torch.stack(images).to(device)
        labels_tensor = torch.tensor(labels).to(device)
        yield images_tensor, labels_tensor

# ResNet 모델을 사용하여 중간 레이어의 특징을 추출
def extract_and_transform_features(images_tensor):
    model = models.resnet50(pretrained=True).to(device)  # 모델을 GPU로 이동
    model.eval()  # 추론 모드로 설정
    
    fc = nn.Linear(2048, 768).to(device)  # FC 레이어도 GPU로 이동
    
    with torch.no_grad():
        # ResNet의 전체 네트워크를 통과시킴
        features = model.conv1(images_tensor)
        features = model.bn1(features)
        features = model.relu(features)
        features = model.maxpool(features)
        features = model.layer1(features)
        features = model.layer2(features)
        features = model.layer3(features)
        features = model.layer4(features)  # layer4를 사용하여 2048차원 벡터 추출
        features = model.avgpool(features)  # 평균 풀링 적용
        features = torch.flatten(features, 1)  # 2048차원 벡터로 변환
        
        # 2048차원 특징 벡터를 768차원으로 축소
        features_768 = fc(features)
    
    return features_768

BATCH_SIZE = 16  # 배치 크기 설정

# DataFrame 저장 준비
all_features = []
all_labels = []

# 배치 단위로 이미지를 처리하고 특징 벡터를 추출
for images_tensor, labels_tensor in load_images_in_batches(image_folder_path, BATCH_SIZE):
    features_768 = extract_and_transform_features(images_tensor)
    all_features.append(features_768.cpu().numpy())  # CPU로 이동하여 numpy 배열로 변환
    all_labels.append(labels_tensor.cpu().numpy())  # 레이블도 CPU로 이동 후 numpy로 변환
    
    # GPU 메모리 해제
    del images_tensor, labels_tensor, features_768
    torch.cuda.empty_cache()

# DataFrame으로 변환 (768차원의 임베딩 벡터 + 감정 레이블)
features_array = np.concatenate(all_features, axis=0)
labels_array = np.concatenate(all_labels, axis=0)

df = pd.DataFrame(features_array, columns=[f'embedding_{i}' for i in range(768)])
df['emotion_label'] = labels_array.astype(int)  # 레이블은 정수형으로 저장

# CSV 파일로 저장 (텍스트 형식과 동일한 형식)
csv_output_path = 'image_embeddings_with_labels_768_augmented_images.csv'
df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
print(f"CSV 파일이 {csv_output_path}에 저장되었습니다.")


c:\Users\USER\anaconda3\envs\cuda_torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\anaconda3\envs\cuda_torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSV 파일이 image_embeddings_with_labels_768_augmented_images.csv에 저장되었습니다.


In [ ]:
# CSV 파일 불러오기
df = csv_output_path
# 각 감정 레이블의 개수 계산
label_counts = df['emotion_label'].value_counts()

# 결과 출력
print(label_counts)
